In [ ]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
v = pd.read_csv("3B42_600KM/3B42_list_check.csv")
v = v[["GIS_ID", "VMAX"]]
v.VMAX -= v.VMAX.min()
v.VMAX.max()

In [ ]:
a = pd.read_table("3B42_600KM/CSV/ATL_199801_TD_1998072712_400.csv", header = None, sep = ',', engine = "python")

b = a[9:33]
print(b.shape)
b = b.iloc[:, 9:33]
print(b.shape)
b

In [ ]:
train = pd.read_csv("training_data_previous_vmax.csv")
train = train[["file", "VMAX", "VMAX_N06H", "VMAX_N12H"]]
len(train.file)
train.head()

In [ ]:
test = pd.read_csv("test_data_previous_vmax.csv")
#test = test[["file", "VMAX", "VMAX_N06H", "VMAX_N12H"]]
test

In [ ]:
a = pd.read_csv("3B42_600KM/CSV/" + train.file[0], header = None)
a.head()
pvmax = np.array([train.VMAX_N06H[0], train.VMAX_N12H[0]])
pvmax

In [ ]:
train_img = []
train_vmax = []
train_label = []
test_img = []
test_vmax = []
test_label = []
for f in range(len(train.file)) :
    filename = "3B42_600KM/CSV/" + train.file[f]
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (44,44)):
            continue
        temp = temp[9:33]
        temp = temp.iloc[:, 9:33]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.VMAX[f]
        train_label.append(lab)
#         pvmax = np.array([train.VMAX_N06H[f], train.VMAX_N12H[f]])
#         train_vmax.append(pvmax)
    except Exception as e:
        pass

for f in range(len(test.file)) :
    filename = "3B42_600KM/CSV/" + test.file[f]
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (44,44)):
            continue
        temp = temp[9:33]
        temp = temp.iloc[:, 9:33]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.VMAX[f]
        test_label.append(lab)
#         pvmax = np.array([test.VMAX_N06H[f], test.VMAX_N12H[f]])
#         test_vmax.append(pvmax)
    except Exception as e:
        pass

In [ ]:
print(len(train_img))
print(len(train_vmax))
print(len(train_label))
print(len(test_img))
print(len(test_vmax))
print(len(test_label))

In [ ]:
type(train_vmax)

In [ ]:
X_train_img = train_img
X_train_vmax = train_vmax
y_train = train_label
X_test_img = test_img
X_test_vmax = test_vmax
y_test = test_label

In [ ]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,24,24,1)
X_train_img = X_train_img.astype('float32')
X_train_vmax = np.array(X_train_vmax)
X_train_vmax = X_train_vmax.reshape(-1,2)
y_train = np.array(y_train)

In [ ]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,24,24,1)
X_test_img = X_test_img.astype('float32')
X_test_vmax = np.array(X_test_vmax)
X_test_vmax = X_test_vmax.reshape(-1,2)
y_test = np.array(y_test)

In [ ]:
import tensorflow as tf 
from tensorflow import keras

In [ ]:
model1 = keras.Sequential([
   keras.layers.Conv2D(64,5, activation = 'relu', input_shape = (24,24,1)),
   keras.layers.Conv2D(64,5, activation = 'relu'),
   keras.layers.Conv2D(64,1, activation = 'relu'),
   keras.layers.MaxPool2D((2,2), strides = 2),
   keras.layers.BatchNormalization(),
   keras.layers.Conv2D(64, 4, activation='relu'),
   keras.layers.Conv2D(64, 4, activation='relu'),
   keras.layers.Conv2D(256,1, activation = 'relu'),
   keras.layers.BatchNormalization(),
   keras.layers.Flatten(),
   keras.layers.Dense(256),
   keras.layers.Dense(165)
])

model2 = keras.Sequential([
   keras.layers.Conv2D(256, 5, activation='relu', input_shape=(24,24,1)),
   keras.layers.MaxPool2D(2,2),
   keras.layers.BatchNormalization(),
   keras.layers.Conv2D(128, 1, activation = 'relu'),
   keras.layers.Conv2D(128,3, activation = 'relu'), 
   keras.layers.MaxPool2D(2,2),
   keras.layers.BatchNormalization(),
   keras.layers.Conv2D(64, 1, activation = 'relu'),
   keras.layers.Conv2D(64,2, activation = 'relu'), 
   keras.layers.MaxPool2D(2,2),
   keras.layers.BatchNormalization(),
   keras.layers.Flatten(),
   keras.layers.Dense(165, activation = 'linear')
])
model3 = keras.Sequential([
    keras.layers.Conv2D(256, (5,2), activation='relu', input_shape=(24,24,1)),
    keras.layers.Conv2D(256, (2,5), activation = 'relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128,3, activation = 'relu'), 
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64,2, activation = 'relu'), 
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(165)
])
model4 = keras.Sequential([
    keras.layers.Conv2D(256, (5,5), activation = 'relu', input_shape = (24,24,1)), 
    keras.layers.MaxPool2D(2,2), 
    keras.layers.Flatten(), 
    keras.layers.Dense(165)
])


model1.summary()
model2.summary()
model3.summary()
model4.summary()

In [ ]:
vmax_input = keras.Input(shape =(2,), name = "vmax_layer")
img_input = keras.Input(shape =(24, 24, 1), name = "img_layer")

w = keras.layers.Conv2D(64,5) (img_input)
w = keras.layers.Conv2D(64,5)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 4)(w)
w = keras.layers.Conv2D(64, 4)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, vmax_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 5) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,3)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, vmax_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (5,2))(img_input)
y = keras.layers.Conv2D(256, (2,5))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,2)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, vmax_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, (5,5))(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, vmax_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

In [ ]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
model4.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [ ]:
model3.fit(X_train_img, y_train, epochs=3, batch_size=32, validation_split = 0.1)
res = model3.evaluate(X_test_img, y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))

In [ ]:
new_model4.fit([X_train_img, X_train_vmax], y_train, epochs=1, batch_size=16, validation_split = 0.1)
res = new_model4.evaluate([X_test_img, X_test_vmax], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))

In [ ]:
new_model4.fit([X_train_img, X_train_vmax], y_train, epochs=1, batch_size=16, validation_split = 0.1)
res = new_model4.evaluate([X_test_img, X_test_vmax], y_test)

In [ ]:
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))
preds = model1.predict(X_test_img)

In [ ]:
x = np.average(preds, axis = 1)
x.size

In [ ]:
x

In [ ]:
test['model4_preds'] = x
test

In [ ]:
np.mean(abs(test.VMAX-test.model4_preds))

In [ ]:
test.to_csv("~/Documents/Python/predicted_results.csv")

In [ ]:
tf.__version__